# Four Arithmetic Deep Neural Network  
Challenge training four arithmetic operation with deep neural network.  
This NN compute within three-digits number without zero.  
NN architecture image below.  

<img src="four_arithmetic_image.png">

## Create Dataset

In [1]:
import numpy as np

X_train = np.random.randint(1, 100, size=(1000000, 2))
X_test = np.random.randint(1, 100, size=(10000, 2))
y_train_add = np.sum(X_train, axis=1)
y_test_add = np.sum(X_test, axis=1)
y_train_sub = np.subtract(X_train[:, 0], X_train[:, 1])
y_test_sub = np.subtract(X_test[:, 0], X_test[:, 1])
y_train_mul = np.multiply(X_train[:, 0], X_train[:, 1])
y_test_mul = np.multiply(X_test[:, 0], X_test[:, 1])
y_train_div = np.divide(X_train[:, 0], X_train[:, 1])
y_test_div = np.divide(X_test[:, 0], X_test[:, 1])
train_data = np.c_[X_train, y_train_add, y_train_sub, y_train_mul, y_train_div]
test_data = np.c_[X_test, y_test_add, y_test_sub, y_test_mul, y_test_div]
y_train, y_test = train_data[:, 2:], test_data[:, 2:]

## Constract NN

In [2]:
import tensorflow as tf

In [10]:
n_inputs = 2
n_hidden1 = 16
n_hidden_add = 32
n_hidden_sub = 32
n_hidden_mul = 32
n_hidden_div = 46
n_hidden2 = 64
n_outputs = 4
dropout_rate = 0.4

In [11]:
def fetch_data(X, y, batch_size):
    n, _ = X.shape
    batch_index = np.random.randint(0, n, size=batch_size)
    X_batch, y_batch = X[batch_index], y[batch_index]
    return X_batch, y_batch

In [12]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=(None, 2), name='X')
y = tf.placeholder(tf.float32, shape=(None, 4), name='y')
training = tf.placeholder_with_default(False, shape=(), name='training')
with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.leaky_relu, name='hidden1')
    add = tf.layers.dense(hidden1, n_hidden_add, activation=tf.nn.leaky_relu, name='add')    
    sub = tf.layers.dense(hidden1, n_hidden_sub, activation=tf.nn.leaky_relu, name='sub')
    mul = tf.layers.dense(hidden1, n_hidden_mul, activation=tf.nn.leaky_relu, name='mul')
    div = tf.layers.dense(hidden1, n_hidden_div, activation=tf.nn.leaky_relu, name='div')
    four_concat = tf.concat([add, sub, mul, div], 1)
    hidden2 = tf.layers.dense(four_concat, n_hidden2, activation=tf.nn.leaky_relu, name='hidden2')
    #drop_hidden2 = tf.layers.dropout(hidden2, dropout_rate, training=training)
    output = tf.layers.dense(hidden2, n_outputs, name='output')

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.square(output - y), name='loss')
    
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope('eval'):
    mse = tf.reduce_mean(tf.square(output - y), name='mse')

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [13]:
mse_summary = tf.summary.scalar('MSE', mse)
logdir = 'tf_logs/four_arithmetic'
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

## Train NN

In [14]:
n_epochs = 2000
batch_size = 20000
n_batchies = int(X_train.shape[0] // batch_size)

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for batch_cnt in range(n_batchies):
            X_batch, y_batch = fetch_data(X_train, y_train, batch_size=batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
        if epoch % 50 == 0:
            MSE = mse_summary.eval(feed_dict={X: X_train, y: y_train})
            step = epoch * n_batchies + batch_cnt
            file_writer.add_summary(MSE, step)
    test_MSE = mse.eval(feed_dict={X: X_test, y: y_test})
    print('test MSE: ', test_MSE)
    saver.save(sess, 'my_models/four_arithmetic_dnn')
file_writer.close()

test MSE:  21.287643


## Test trained NN

In [15]:
with tf.Session() as sess:
    saver.restore(sess, 'my_models/four_arithmetic_dnn')
    y_pred = output.eval(session=sess, feed_dict={X: X_test})
    pred_set = np.float16(np.c_[X_test, y_pred])
    print(pred_set)

INFO:tensorflow:Restoring parameters from my_models/four_arithmetic_dnn
[[ 7.8000e+01  1.3000e+01  9.1125e+01  6.4938e+01  1.0055e+03  4.9609e+00]
 [ 2.0000e+00  1.0000e+00  6.4336e+00  1.9336e+00 -3.3477e+00  2.3359e+00]
 [ 5.1000e+01  5.6000e+01  1.0706e+02 -4.7773e+00  2.8700e+03  8.0322e-01]
 ...
 [ 8.0000e+01  8.6000e+01  1.6600e+02 -5.6484e+00  6.8840e+03  1.2744e-01]
 [ 2.2000e+01  6.1000e+01  8.2938e+01 -3.9219e+01  1.3400e+03 -4.6045e-01]
 [ 3.7000e+01  9.9000e+01  1.3575e+02 -6.2656e+01  3.6660e+03 -3.2251e-01]]


In [16]:
print(y_test)

[[ 9.10000000e+01  6.50000000e+01  1.01400000e+03  6.00000000e+00]
 [ 3.00000000e+00  1.00000000e+00  2.00000000e+00  2.00000000e+00]
 [ 1.07000000e+02 -5.00000000e+00  2.85600000e+03  9.10714286e-01]
 ...
 [ 1.66000000e+02 -6.00000000e+00  6.88000000e+03  9.30232558e-01]
 [ 8.30000000e+01 -3.90000000e+01  1.34200000e+03  3.60655738e-01]
 [ 1.36000000e+02 -6.20000000e+01  3.66300000e+03  3.73737374e-01]]


## Summary  
Divide is difficult ;(  
Maybe other computings are good.

test concat

In [ ]:
tf.reset_default_graph()
init = tf.global_variables_initializer()

a = tf.constant([[1,2,3], [4,5,6]], dtype=tf.int16)
b = tf.constant([[7,8,9], [10,11,12]], dtype=tf.int16)
c = tf.constant([[13,14,15], [16,17,18]], dtype=tf.int16)
d = tf.constant([[19,20,21], [22,23,24]], dtype=tf.int16)
concat = tf.concat([a,b,c,d], 1)
with tf.Session() as sess:
    init.run()
    con_con = concat.eval()
    print(con_con)